
<a href="https://colab.research.google.com/github/is-leeroy-jenkins/Halo-Kitty-Adventures/blob/main/sql/notebooks/sqlite.ipynb" target="_parent">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




# 🗂️ SQLite SQL — A Comprehensive Guide for Learners and Developers

*A Deep Dive into the SQL Language of the SQLite Engine*

---

## 🧭 Introduction

**SQLite** is a lightweight, server-less, self-contained SQL database engine that stores all its data in a single file.
It implements most of the SQL-92 standard and is used in everything from web browsers and mobile apps to embedded systems and operating-system utilities.

Unlike server databases (SQL Server, MySQL, PostgreSQL), SQLite runs **in-process** with the application — there’s no external service to install or manage.
You simply open a file, execute SQL statements, and the engine handles data persistence, indexing, and transactions automatically.

This guide provides a comprehensive overview of **SQLite’s SQL dialect**, its data model, and its unique implementation details.
All examples use clean SQL syntax with explanatory commentary below each code block.

---

## ⚙️ Understanding SQLite Architecture

SQLite is a **file-based relational database**.
When you connect to a database such as `company.db`, SQLite creates (or opens) a single file on disk containing:

* Tables (data)
* Indexes
* Views
* Triggers
* The database schema

There’s no user authentication, no server configuration, and no networking — the database engine runs entirely in your application process.

SQLite supports both **persistent databases** (`.db` files) and **in-memory databases** (`:memory:`) that exist only during program execution.

---

## 🧱 Basic SQL Structure

The foundation of SQL in SQLite (and all relational systems) is built around four main statement categories:

| Category                             | Purpose                           | Examples                                     |
| ------------------------------------ | --------------------------------- | -------------------------------------------- |
| **DDL (Data Definition Language)**   | Create or alter database objects. | `CREATE TABLE`, `DROP TABLE`, `CREATE INDEX` |
| **DML (Data Manipulation Language)** | Insert, update, or delete data.   | `INSERT`, `UPDATE`, `DELETE`                 |
| **DQL (Data Query Language)**        | Retrieve data.                    | `SELECT`, `WITH`, `JOIN`                     |
| **DCL/Transaction Control**          | Manage changes.                   | `BEGIN`, `COMMIT`, `ROLLBACK`                |

Each section below explains these categories in practical detail.

---

## 📚 Creating Tables and Schemas

Tables store data as rows and columns, similar to spreadsheets but strongly typed and indexed.

```sql
CREATE TABLE Employees (
    EmployeeID   INTEGER PRIMARY KEY,
    FirstName    TEXT NOT NULL,
    LastName     TEXT NOT NULL,
    Department   TEXT,
    HireDate     TEXT DEFAULT CURRENT_DATE,
    Salary       REAL
);
```

**Explanation:**

* `INTEGER PRIMARY KEY` defines a unique identifier. In SQLite, this automatically creates an **alias for the internal rowid** (a 64-bit integer unique to each record).
* `TEXT`, `REAL`, and `INTEGER` are **type affinities**, not rigid types — SQLite stores values dynamically while maintaining data consistency.
* `DEFAULT CURRENT_DATE` automatically inserts the current date on record creation.

### Viewing Tables

List all tables in the database:

```sql
.tables
```

Show the schema of a specific table:

```sql
.schema Employees
```

---

## ✍️ Inserting Data

Add rows using `INSERT INTO`.

```sql
INSERT INTO Employees (FirstName, LastName, Department, Salary)
VALUES ('Jane', 'Doe', 'Finance', 85000);
```

You can also insert multiple rows:

```sql
INSERT INTO Employees (FirstName, LastName, Department, Salary)
VALUES
  ('John', 'Smith', 'HR', 72000),
  ('Alice', 'Brown', 'IT', 95000),
  ('Bob', 'Miller', 'Finance', 78000);
```

SQLite enforces constraints (`NOT NULL`, `UNIQUE`, `PRIMARY KEY`) automatically.

---

## 🔍 Querying Data with SELECT

The **SELECT** statement retrieves data from one or more tables.

```sql
SELECT FirstName, LastName, Department
FROM Employees
WHERE Department = 'Finance'
ORDER BY LastName;
```

**Explanation:**

* `SELECT` specifies the columns to return.
* `FROM` identifies the table.
* `WHERE` filters results.
* `ORDER BY` sorts results alphabetically by `LastName`.

### Using Aliases

```sql
SELECT FirstName || ' ' || LastName AS FullName, Salary AS AnnualPay
FROM Employees;
```

Here, `||` concatenates strings, and `AS` assigns readable column labels.

---

## ⚖️ Filtering Data with WHERE

Use logical and comparison operators to refine results.

| Operator             | Meaning            | Example                                  |
| -------------------- | ------------------ | ---------------------------------------- |
| `=`                  | Equal to           | `WHERE Department = 'IT'`                |
| `<>` or `!=`         | Not equal          | `WHERE Department <> 'Finance'`          |
| `<`, `>`, `<=`, `>=` | Comparison         | `WHERE Salary >= 90000`                  |
| `AND`, `OR`, `NOT`   | Combine conditions | `WHERE Department='IT' AND Salary>80000` |

### NULL Handling

Use `IS NULL` and `IS NOT NULL`:

```sql
SELECT * FROM Employees WHERE Department IS NULL;
```

---

## 🔢 Sorting and Limiting Results

### ORDER BY

```sql
SELECT * FROM Employees
ORDER BY Salary DESC;
```

Sorts salaries from highest to lowest.

### LIMIT and OFFSET

```sql
SELECT * FROM Employees
ORDER BY EmployeeID
LIMIT 5 OFFSET 10;
```

Returns 5 records starting from the 11th row — useful for pagination.

---
Excellent catch — and very perceptive.

No, the SQLite tutorial above does **not yet include the logical execution order** (sometimes called **query evaluation order**) for `SELECT` statements and other SQL clauses.

That’s a fundamental concept worth adding, because understanding **how SQLite processes queries internally** (the *logical* and *physical* order of execution) makes it easier to write efficient, correct SQL.

Let’s append that section now in the same Markdown style as the tutorial.
You can paste it directly after the **“🔍 Querying Data with SELECT”** section or near the end of the document under a heading like **“🧠 Understanding SQL Execution Order.”**

---

## 🧠 Understanding SQL Execution Order in SQLite

SQL queries — especially `SELECT` — are **not executed in the same order they’re written**.
SQLite (and most relational engines) follow a **logical processing order**, meaning it conceptually evaluates one clause at a time to build the final result set.

This order helps explain why certain elements (like aliases or aggregates) can’t be referenced in earlier clauses.

---

### Logical Order of Execution

| Step  | Clause           | Purpose                                                      |
| ----- | ---------------- | ------------------------------------------------------------ |
| **1** | `FROM`           | Identify the tables and perform joins or subqueries.         |
| **2** | `WHERE`          | Filter rows based on given conditions (row-level filtering). |
| **3** | `GROUP BY`       | Group rows with matching values into summary groups.         |
| **4** | `HAVING`         | Filter the grouped data (aggregate-level filtering).         |
| **5** | `SELECT`         | Choose which columns or expressions to return.               |
| **6** | `DISTINCT`       | Remove duplicate rows from the result set.                   |
| **7** | `ORDER BY`       | Sort the final results.                                      |
| **8** | `LIMIT / OFFSET` | Restrict the number of rows returned.                        |

---

### Example: Understanding Clause Order

Consider this query:

```sql
SELECT Department, AVG(Salary) AS AvgSalary
FROM Employees
WHERE HireDate >= '2020-01-01'
GROUP BY Department
HAVING AVG(Salary) > 85000
ORDER BY AvgSalary DESC
LIMIT 5;
```

**Execution flow:**

1. **FROM** — SQLite loads the `Employees` table.
2. **WHERE** — Filters rows to only those hired after 2020.
3. **GROUP BY** — Groups remaining rows by `Department`.
4. **HAVING** — Keeps only departments where `AVG(Salary) > 85000`.
5. **SELECT** — Projects two columns: `Department` and the aggregate `AVG(Salary)`.
6. **ORDER BY** — Sorts departments by `AvgSalary` descending.
7. **LIMIT** — Returns the top five departments.

---

### Key Insights

* The **`FROM`** and **`WHERE`** clauses act first — they determine *which rows* exist for grouping or aggregation.
* **`SELECT`** actually happens *late*, which is why you can’t use column aliases in the `WHERE` clause.
* **Aggregates (SUM, AVG, COUNT, etc.)** can only be used after grouping has occurred.
* **`ORDER BY`** and **`LIMIT`** apply to the *final result set*, not to individual groups.

---

### Physical vs. Logical Order

The above order describes **logical processing**, not necessarily the physical sequence of operations inside SQLite’s query planner.
Internally, SQLite may:

* Optimize joins
* Reorder filters
* Use indexes
  to execute queries more efficiently.
  However, the *logical model* above remains the conceptual blueprint you should rely on when reasoning about query behavior.

---

### Quick Reference Diagram

```
┌─────────────────────────────┐
│ FROM → WHERE → GROUP BY     │
│ → HAVING → SELECT → ORDER BY│
│ → LIMIT / OFFSET            │
└─────────────────────────────┘
```

---

### Why This Matters

Understanding clause order clarifies many common SQL frustrations:

| Common Confusion                                   | Explanation                                                            |
| -------------------------------------------------- | ---------------------------------------------------------------------- |
| “Why can’t I use SELECT alias in WHERE?”           | Because `WHERE` executes before `SELECT`.                              |
| “Why can’t I filter aggregate results with WHERE?” | Because aggregates don’t exist until `GROUP BY` and must use `HAVING`. |
| “Why does ORDER BY see aliases?”                   | Because `ORDER BY` executes *after* `SELECT`, when aliases exist.      |

---

## 🔗 Joining Tables

SQLite fully supports ANSI join syntax.

### INNER JOIN

```sql
SELECT e.FirstName, e.LastName, d.DepartmentName
FROM Employees AS e
INNER JOIN Departments AS d
ON e.Department = d.DepartmentName;
```

### LEFT JOIN

```sql
SELECT c.CustomerName, o.OrderID
FROM Customers AS c
LEFT JOIN Orders AS o
ON c.CustomerID = o.CustomerID;
```

SQLite does not implement `RIGHT JOIN` or `FULL OUTER JOIN` directly.
Use `UNION` of `LEFT JOIN` and `RIGHT JOIN` patterns to simulate them if needed.

---

## 🧮 Aggregate Functions and Grouping

SQLite provides standard aggregate functions:

| Function          | Description     | Example         |
| ----------------- | --------------- | --------------- |
| `COUNT()`         | Counts rows     | `COUNT(*)`      |
| `SUM()`           | Adds values     | `SUM(Salary)`   |
| `AVG()`           | Averages values | `AVG(Salary)`   |
| `MIN()` / `MAX()` | Finds min/max   | `MIN(HireDate)` |

### GROUP BY Example

```sql
SELECT Department, AVG(Salary) AS AvgSalary
FROM Employees
GROUP BY Department
HAVING AVG(Salary) > 80000;
```

`HAVING` filters groups after aggregation; `WHERE` filters rows before grouping.

---

## 🧩 Subqueries

Subqueries can appear in `WHERE`, `FROM`, or `SELECT` clauses.

### Using IN

```sql
SELECT FirstName, LastName
FROM Employees
WHERE Department IN (
    SELECT DepartmentName FROM Departments WHERE Location = 'HQ'
);
```

### Using EXISTS

```sql
SELECT c.CustomerName
FROM Customers AS c
WHERE EXISTS (
    SELECT 1 FROM Orders AS o WHERE o.CustomerID = c.CustomerID
);
```

Subqueries are re-evaluated for each row unless optimized by the query planner; indexes improve performance dramatically.

---

## ⚙️ Updating and Deleting Data

### UPDATE

```sql
UPDATE Employees
SET Salary = Salary * 1.10
WHERE Department = 'Sales';
```

### DELETE

```sql
DELETE FROM Employees
WHERE HireDate < '2020-01-01';
```

### REPLACE

SQLite’s `REPLACE INTO` behaves like an `INSERT` that overwrites existing rows with matching primary keys.

```sql
REPLACE INTO Employees (EmployeeID, FirstName, LastName, Department, Salary)
VALUES (3, 'Alice', 'Brown', 'IT', 98000);
```

---

## 🧱 Creating and Managing Indexes

Indexes accelerate lookups and joins.

```sql
CREATE INDEX idx_employees_department
ON Employees (Department);
```

Delete an index:

```sql
DROP INDEX idx_employees_department;
```

SQLite automatically creates indexes for primary keys and unique constraints.

---

## 🧮 Constraints and Keys

SQLite supports the main integrity constraints:

| Constraint      | Purpose                | Example                                          |
| --------------- | ---------------------- | ------------------------------------------------ |
| **PRIMARY KEY** | Unique identifier      | `EmployeeID INTEGER PRIMARY KEY`                 |
| **UNIQUE**      | Prevents duplicates    | `UNIQUE(Email)`                                  |
| **NOT NULL**    | Disallows NULL values  | `LastName TEXT NOT NULL`                         |
| **CHECK**       | Validates data         | `CHECK(Salary > 0)`                              |
| **FOREIGN KEY** | Enforces relationships | `FOREIGN KEY(DeptID) REFERENCES Departments(ID)` |

### Enabling Foreign Keys

Foreign-key enforcement is disabled by default. Enable it per session:

```sql
PRAGMA foreign_keys = ON;
```

---

## 🧾 Views

A **view** is a saved query that behaves like a virtual table.

```sql
CREATE VIEW vw_HighEarners AS
SELECT FirstName, LastName, Department, Salary
FROM Employees
WHERE Salary > 90000;
```

Use it like a normal table:

```sql
SELECT * FROM vw_HighEarners;
```

Delete it with:

```sql
DROP VIEW vw_HighEarners;
```

Views do not store data — they re-execute their underlying SQL each time they’re queried.

---

## 🔄 Transactions

Transactions ensure multiple changes occur together or not at all.

```sql
BEGIN TRANSACTION;

UPDATE Accounts SET Balance = Balance - 500 WHERE AccountID = 1;
UPDATE Accounts SET Balance = Balance + 500 WHERE AccountID = 2;

COMMIT;
```

If any statement fails, use `ROLLBACK` to undo all operations.

SQLite automatically commits each statement unless wrapped in a transaction block.

---

## 🧩 Triggers

Triggers automatically execute SQL in response to changes.

```sql
CREATE TRIGGER trg_UpdateAudit
AFTER UPDATE ON Employees
FOR EACH ROW
BEGIN
    INSERT INTO AuditLog (Action, TableName, RecordID, Timestamp)
    VALUES ('UPDATE', 'Employees', OLD.EmployeeID, datetime('now'));
END;
```

Drop a trigger:

```sql
DROP TRIGGER trg_UpdateAudit;
```

Triggers are ideal for auditing, enforcing rules, and maintaining consistency.

---

## 🧰 Built-In SQLite Functions

SQLite offers a rich set of built-in scalar and aggregate functions.

| Category      | Function                     | Example            | Description               |
| ------------- | ---------------------------- | ------------------ | ------------------------- |
| **String**    | `LOWER(text)`, `UPPER(text)` | `UPPER(LastName)`  | Case conversion           |
|               | `TRIM(text)`                 | `TRIM(Name)`       | Removes spaces            |
|               | `SUBSTR(text,start,len)`     | `SUBSTR(Name,1,3)` | Extract substring         |
| **Date/Time** | `date('now')`                |                    | Current date              |
|               | `datetime('now','+7 days')`  |                    | Future date/time          |
|               | `strftime('%Y-%m',date)`     |                    | Custom formatting         |
| **Math**      | `ABS(x)`                     |                    | Absolute value            |
|               | `ROUND(x, n)`                |                    | Round to n decimals       |
|               | `RANDOM()`                   |                    | Random integer            |
| **Aggregate** | `GROUP_CONCAT(expr, sep)`    |                    | Concatenates group values |

SQLite supports user-defined functions in external applications, but built-ins cover most use cases.

---

## 🧮 The WITH Clause (Common Table Expressions)

CTEs simplify complex queries by creating temporary named result sets.

```sql
WITH DeptAverage AS (
    SELECT Department, AVG(Salary) AS AvgSalary
    FROM Employees
    GROUP BY Department
)
SELECT e.FirstName, e.LastName, e.Salary, d.AvgSalary
FROM Employees AS e
JOIN DeptAverage AS d ON e.Department = d.Department
WHERE e.Salary > d.AvgSalary;
```

This query lists employees earning above their departmental average.

---

## 📊 The PRAGMA Command

`PRAGMA` statements control SQLite’s internal behavior or expose metadata.

Examples:

```sql
PRAGMA foreign_keys = ON;
PRAGMA table_info(Employees);
PRAGMA database_list;
PRAGMA encoding;
```

Each pragma is specific to SQLite and often acts like a specialized function call.

---

## ⚡ Backup, Export, and Import

### Exporting Data

```sql
.mode csv
.output employees.csv
SELECT * FROM Employees;
.output stdout
```

### Importing Data

```sql
.mode csv
.import employees.csv Employees
```

These commands are used in the **sqlite3 command-line shell**.

---

## ⚠️ Common Pitfalls and Best Practices

| Issue                           | Recommendation                                                                               |
| ------------------------------- | -------------------------------------------------------------------------------------------- |
| **Dynamic Typing**              | SQLite stores values flexibly — use CHECK constraints for strict validation.                 |
| **Foreign Keys Off by Default** | Always enable with `PRAGMA foreign_keys=ON;`.                                                |
| **Date Handling**               | Dates are stored as text — use `strftime()` for manipulation.                                |
| **Transactions**                | Use explicit transactions for batch inserts to improve performance.                          |
| **NULLs in Aggregates**         | Functions ignore NULLs; use `COALESCE()` to replace them.                                    |
| **Case Sensitivity**            | By default, text comparisons are case-insensitive; use `COLLATE BINARY` for strict matching. |

---

## 🧾 Quick Reference Tables

### Data Type Affinities

| Declared Type | Storage Class  | Typical Use               |
| ------------- | -------------- | ------------------------- |
| `INTEGER`     | Integer        | Whole numbers, IDs        |
| `REAL`        | Floating-point | Decimal values            |
| `TEXT`        | Text string    | Names, descriptions       |
| `BLOB`        | Binary         | Images, files             |
| `NUMERIC`     | Flexible       | Dates, booleans, currency |

SQLite uses **type affinity** rather than strict typing — meaning any column can technically store any value, but conversion rules preserve intent.

---

### Common Date/Time Functions

| Function                   | Description        | Example Result        |
| -------------------------- | ------------------ | --------------------- |
| `date('now')`              | Current date       | `2025-11-06`          |
| `datetime('now','-1 day')` | One day ago        | `2025-11-05 08:00:00` |
| `strftime('%Y', date)`     | Extract year       | `2025`                |
| `julianday('now')`         | Days since 4713 BC | `2461428.5`           |

---

## 🧩 Summary

SQLite’s SQL implementation is both **compact and powerful**.
It supports nearly the entire SQL-92 language, plus unique extensions for embedded use cases.

Key takeaways:

* SQLite is **serverless** — everything lives in one file.
* It uses **dynamic typing** and **type affinities** rather than rigid data types.
* Supports **transactions, joins, triggers, and views** fully.
* Ideal for lightweight applications, local storage, and data interchange.

Mastering SQLite SQL prepares you to design efficient, portable databases for any environment — from desktop tools to mobile apps.

---

### 📚 Further Reading

* **Official SQLite Documentation:** [https://www.sqlite.org/docs.html](https://www.sqlite.org/docs.html)
* **SQLite SQL Syntax:** [https://www.sqlite.org/lang.html](https://www.sqlite.org/lang.html)
* **“Using SQLite” by Jay A. Kreibich (O’Reilly, 2010)** — an authoritative reference

---
